In [2]:
import numpy as np
import pandas as pd

df_viajes= pd.read_csv("C:/Users/david/Downloads/dataset_viajes.csv")

In [ ]:
#Aquí podemos ver qué valores nulos tenemos

In [3]:
df_viajes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Id_vuelo    1000 non-null   object 
 1   Aircompany  1000 non-null   object 
 2   Origen      1000 non-null   object 
 3   Destino     1000 non-null   object 
 4   Distancia   872 non-null    float64
 5   avion       1000 non-null   object 
 6   consumo_kg  862 non-null    object 
 7   duracion    853 non-null    float64
dtypes: float64(2), object(6)
memory usage: 62.6+ KB


# Clasificación

Por su origen, y dificultad para "recuperarlos", los datos faltantes se pueden clasificar en tres categorías:

1. **Missing Completely at Random (MCAR).**  
   Estos son datos que se pierden de verdad de forma esporádica y aleatoria. La pérdida de datos no tiene que ver con la observación estudiada. Por ejemplo, un sensor que se quede sin batería, un cuestionario perdido en una oficina de correos, o una muestra sanguínea fallida en un laboratorio. En general, como es tan aleatoria la pérdida, si no es masiva, son datos que se pueden recuperar o estimar a partir de otras "filas" con datos similares (los que rellenamos con las medidas, etc. de otros campos)

2. **Missing at Random (MAR).**  
   El hecho de la pérdida está relacionado con otra variable. Por ejemplo, teléfonos que se estropean y ya no podemos medir su velocidad de acceso a internet. Podemos recuperar esos datos mirando teléfonos similares que no se estropean; es peor que el anterior pero aún podemos tratarlo.

3. **Missing not at Random (MNAR).**  
   Datos incompletos que no se explican por motivos anteriores y que en general no podemos recuperar (estos son candidatos a que los borremos).

En general, si tienes muchos datos, tira los *missings*, no pierdas el tiempo. Sólo si tus datos son escasos y estás en modo EDA (Exploratory Data Analysis), intenta recuperarlos, pero ojo: si tienes *missings*, pocos datos y crees que son valiosos los que has perdido, intenta generar nuevos o recuperarlos, pero no estimarlos (para Machine Learning, estimar sobre lo que ya existe y luego usarlo como si fuera verdad, es una forma de introducir sesgos "peligrosos", véase ChatGPT

# Aproximaciones para tratar los missings

## A. Intenta obtenerlos

- A veces es posible encontrar los valores incompletos (repitiendo una encuesta, buscando en otras fuentes, etc.). Esto no suele ser lo habitual.

En nuestro dataset era posible para las distancias, ya que teniendo Origen y Destino la distancia no cambia y podemos completar esos campos faltantes.

In [9]:
df_viajes_sin_na = df_viajes.copy()
df_viajes_sin_na["Destino"] = df_viajes_sin_na.Destino.str.lower().str.capitalize()

df_viajes_sin_na["Distancia_Corregida"]= df_viajes_sin_na.groupby(["Origen","Destino"])["Distancia"].transform("mean")

In [10]:
df_viajes_sin_na.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Id_vuelo             1000 non-null   object 
 1   Aircompany           1000 non-null   object 
 2   Origen               1000 non-null   object 
 3   Destino              1000 non-null   object 
 4   Distancia            872 non-null    float64
 5   avion                1000 non-null   object 
 6   consumo_kg           862 non-null    object 
 7   duracion             853 non-null    float64
 8   Distancia_Corregida  1000 non-null   float64
dtypes: float64(3), object(6)
memory usage: 70.4+ KB


# Descartar datos, es decir las filas

Omitir los registros (filas) con algún dato faltante y analizar el dataset resultante. Si el tamaño del conjunto de datos es grande, y no hay demasiados *missing values*, puede ser una estrategia válida. Sin embargo, cuando no tenemos muchos datos o no se satisface MCAR, no es la mejor aproximación, y puede causar sesgo en los datos.

Aún así, cómo hacerlo con Pandas: recordamos que en la práctica nuestro criterio fue descartar las filas que tenían *missing* en los tres campos, ¿por qué? Porque tenían demasiado "error"; rellenarlos con estimaciones (medias, modas, modelos de IA, etc.) no tiene sentido porque convierte al dato en demasiado "artificial". Hagámoslo:

In [11]:
df_viajes_sin_na_I= df_viajes.dropna(subset=["Distancia","consumo_kg","duracion"],how="all")

In [12]:
df_viajes_sin_na_I.info()

<class 'pandas.core.frame.DataFrame'>
Index: 997 entries, 0 to 999
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Id_vuelo    997 non-null    object 
 1   Aircompany  997 non-null    object 
 2   Origen      997 non-null    object 
 3   Destino     997 non-null    object 
 4   Distancia   872 non-null    float64
 5   avion       997 non-null    object 
 6   consumo_kg  862 non-null    object 
 7   duracion    853 non-null    float64
dtypes: float64(2), object(6)
memory usage: 70.1+ KB


# Eliminar campos

Si una variable tiene muchos *missings*, una opción puede ser eliminar la columna del dataset. Por ejemplo, una variable con el 99% de nulos, no aportará mucha información y podremos eliminarla. En cualquier caso, es una decisión que hay que tomar con cuidado, y depende de cada caso.

Para saber cuál es la proporción de nulos, acudíamos a:

In [16]:
df_viajes["duracion"].value_counts(normalize=True, dropna=False)[np.nan]

np.float64(0.147)

In [17]:
df_viajes["consumo_kg"].value_counts(normalize=True, dropna=False)[np.nan]

np.float64(0.138)

In [18]:
df_viajes["Distancia"].value_counts(normalize=True, dropna=False)[np.nan]

np.float64(0.128)

Son porcentajes muy bajos, es decir, perdemos mucha información (más del 85% de los datos) si nos deshacemos de la columna; el valor de la columna ya es otra cosa. Pero si aún así quisieramos eliminar esas columnas, usaríamos `drop`:

In [19]:
df_viajes_sin_na_II = df_viajes.drop(columns=["consumo_kg"])

In [21]:
df_viajes_sin_na_II.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Id_vuelo    1000 non-null   object 
 1   Aircompany  1000 non-null   object 
 2   Origen      1000 non-null   object 
 3   Destino     1000 non-null   object 
 4   Distancia   872 non-null    float64
 5   avion       1000 non-null   object 
 6   duracion    853 non-null    float64
dtypes: float64(2), object(5)
memory usage: 54.8+ KB


# Media, Mediana y Moda

En lugar de eliminar, reemplazamos valores *missing* con estimaciones estadísticas como la media, la moda o la mediana. En una sustitución por la media, el valor medio de una variable se usa en lugar del valor de los datos que faltan para esa misma variable. Esto tiene la ventaja de no cambiar la media muestral de esa variable. Sin embargo, con valores faltantes que no son estrictamente aleatorios, especialmente en presencia de una gran desigualdad en el número de valores faltantes para las diferentes variables, el método de sustitución de medias puede conducir a un sesgo inconsistente.

Este es el método que sugeríamos en las unidades dedicadas a Pandas, pero ten en cuenta que, como se dice anteriormente, se introduce sesgo. En cualquier caso, y como vimos en la práctica, se puede hilar fino empleando la media de agrupaciones.

In [22]:
df_viajes_sin_na_IV = df_viajes.copy()
df_viajes_sin_na_IV["consumo_kg"] = df_viajes_sin_na_IV["consumo_kg"].str.replace(",", "").astype("float")
df_viajes_sin_na_IV["consumo_kg_medio"] = df_viajes_sin_na_IV.groupby(["Aircompany", "avion"])["consumo_kg"].transform("mean")
df_viajes_sin_na_IV.loc[df_viajes_sin_na_IV.consumo_kg.isna(), "consumo_kg"] = df_viajes_sin_na_IV.loc[df_viajes_sin_na_IV.consumo_kg.isna(), "consumo_kg_medio"]
df_viajes_sin_na_IV.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Id_vuelo          1000 non-null   object 
 1   Aircompany        1000 non-null   object 
 2   Origen            1000 non-null   object 
 3   Destino           1000 non-null   object 
 4   Distancia         872 non-null    float64
 5   avion             1000 non-null   object 
 6   consumo_kg        1000 non-null   float64
 7   duracion          853 non-null    float64
 8   consumo_kg_medio  1000 non-null   float64
dtypes: float64(4), object(5)
memory usage: 70.4+ KB


# Añadir variable binaria indicando NaNs

Como complemento a estimar los valores perdidos o NaN, podemos capturar el hecho de que es un *missing* re-estimado creando una variable binaria adicional indicando si era un valor *missing* (1, True) o no (0, False). Esto nos permitirá descartarlos en casos que no queramos contar con ellos o tenerlos en cuenta en casos que sí, con solo filtrar o no por ese campo adicional.

**IMPORTANTE:** Esto se tiene que hacer antes de reimputar o estimar los *missing*, después ya no se sabrá cuáles lo eran o no. O tratar con una copia, como hemos hecho nosotros.

In [23]:
df_viajes_sin_na_IV["Era_missing_consumo_kg"]=df_viajes.consumo_kg.isna()

In [24]:
df_viajes_sin_na_IV.loc[df_viajes_sin_na_IV.Era_missing_consumo_kg==False]

,Id_vuelo,Aircompany,Origen,Destino,Distancia,avion,consumo_kg,duracion,consumo_kg_medio,Era_missing_consumo_kg
1,Fly_BaRo_10737,FlyQ,Bali,Roma,12738.0,Boeing 737,3.347913e+04,1167.0,4.561165e+06,False
3,Mol_PaCi_10737,MoldaviAir,París,Cincinnati,6370.0,Boeing 737,1.702701e+04,503.0,2.562361e+14,False
4,Tab_CiRo_10747,TabarAir,Cincinnati,Roma,7480.0,Boeing 747,8.611574e+04,518.0,4.799067e+14,False
5,Mol_CaMe_10737,MoldaviAir,Cádiz,Melbourne,20029.0,Boeing 737,5.314815e+04,NaN,2.562361e+14,False
6,Mol_PaLo_11320,MoldaviAir,París,Londres,344.0,Airbus A320,9.152464e+02,44.0,6.165621e+08,False
...,...,...,...,...,...,...,...,...,...,...
995,Pam_LoNu_10747,PamPangea,Londres,Nueva York,5566.0,Boeing 747,6.230024e+07,391.0,1.145966e+14,False
996,Mol_MeLo_10747,MoldaviAir,Melbourne,Londres,16900.0,Boeing 747,1.948546e+05,1326.0,5.269108e+07,False
997,Mol_BaPa_10747,MoldaviAir,Bali,París,11980.0,Boeing 747,1.289839e+05,818.0,5.269108e+07,False
998,Air_CaCi_10747,Airnar,Cádiz,Cincinnati,6624.0,Boeing 747,7.202408e+04,461.0,4.155047e+07,False
